In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../Data/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:
df['FastingBS'].nunique()

2

### Handle numerical attributes

In [7]:
heart_num = df[['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak', 'FastingBS']]
heart_num.head()

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,FastingBS
0,40,140,289,172,0.0,0
1,49,160,180,156,1.0,0
2,37,130,283,98,0.0,0
3,48,138,214,108,1.5,0
4,54,150,195,122,0.0,0


In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

heart_num_np = np.array(heart_num)
heart_num_np_scaled = scaler.fit_transform(heart_num_np)
print(heart_num_np.shape, heart_num_np_scaled.shape)
print(heart_num_np[0], '\n', heart_num_np_scaled[0])

(918, 6) (918, 6)
[ 40. 140. 289. 172.   0.   0.] 
 [-1.4331398   0.41090889  0.82507026  1.38292822 -0.83243239 -0.55134134]


### Handle Categorial Attributes

In [9]:
heart_cat = df[['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']]
heart_cat.head()

,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope
0,M,ATA,Normal,N,Up
1,F,NAP,Normal,N,Flat
2,M,ATA,ST,N,Up
3,F,ASY,Normal,Y,Flat
4,M,NAP,Normal,N,Up


In [27]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

heart_cat_np = np.array(heart_cat)
heart_cat_np_encoded = cat_encoder.fit_transform(heart_cat_np)
heart_cat_np_encoded = heart_cat_np_encoded.toarray()
print(heart_cat_np.shape, heart_cat_np_encoded.shape)
print(heart_cat_np[0], '\n', heart_cat_np_encoded[0])
print(type(heart_cat_np_encoded))
print(cat_encoder.get_feature_names_out())

(918, 5) (918, 14)
['M' 'ATA' 'Normal' 'N' 'Up'] 
 [0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1.]
<class 'numpy.ndarray'>
['x0_F' 'x0_M' 'x1_ASY' 'x1_ATA' 'x1_NAP' 'x1_TA' 'x2_LVH' 'x2_Normal'
 'x2_ST' 'x3_N' 'x3_Y' 'x4_Down' 'x4_Flat' 'x4_Up']


In [26]:
heart_cat_num_full = np.hstack((heart_cat_np_encoded, heart_num_np_scaled))
type(heart_cat_num_full)

numpy.ndarray

In [37]:
df_new = pd.DataFrame(heart_cat_num_full, columns=[np.hstack((cat_encoder.get_feature_names_out(),
                                                             ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak', 'FastingBS']))])

In [41]:
df_new = df_new.assign(HeartDisease=np.array(df['HeartDisease']))
df_new.head()

,x0_F,x0_M,x1_ASY,x1_ATA,x1_NAP,x1_TA,x2_LVH,x2_Normal,x2_ST,x3_N,...,x4_Down,x4_Flat,x4_Up,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,FastingBS,HeartDisease
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,-1.433140,0.410909,0.825070,1.382928,-0.832432,-0.551341,0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,-0.478484,1.491752,-0.171961,0.754157,0.105664,-0.551341,1
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,-1.751359,-0.129513,0.770188,-1.525138,-0.832432,-0.551341,0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,-0.584556,0.302825,0.139040,-1.132156,0.574711,-0.551341,1
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.051881,0.951331,-0.034755,-0.581981,-0.832432,-0.551341,0


In [42]:
df_new.to_csv("heart_processed.csv", index=False)

In [43]:
df2 = pd.read_csv("heart_processed.csv")
df2.head()

,x0_F,x0_M,x1_ASY,x1_ATA,x1_NAP,x1_TA,x2_LVH,x2_Normal,x2_ST,x3_N,...,x4_Down,x4_Flat,x4_Up,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,FastingBS,HeartDisease
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,-1.433140,0.410909,0.825070,1.382928,-0.832432,-0.551341,0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,-0.478484,1.491752,-0.171961,0.754157,0.105664,-0.551341,1
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,-1.751359,-0.129513,0.770188,-1.525138,-0.832432,-0.551341,0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,-0.584556,0.302825,0.139040,-1.132156,0.574711,-0.551341,1
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.051881,0.951331,-0.034755,-0.581981,-0.832432,-0.551341,0
